In [1]:
from __future__ import print_function, division

from PIL import Image

from keras.datasets import mnist

from keras.layers import Input, Dense, Reshape, Flatten, Dropout

from keras.layers import BatchNormalization, Activation, ZeroPadding2D

from keras.layers.advanced_activations import LeakyReLU

from keras.layers.convolutional import UpSampling2D, Conv2D

from keras.models import Sequential, Model

from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

import numpy as np

from numpy import random

import scipy

from scipy import misc

from PIL import Image

import matplotlib.pyplot as plt

from scipy import ndimage

import os


os.chdir(r'C:\\users\\sarva\\Projects\\GAN_face\\')



class GAN():

    def __init__(self):

        self.img_rows = 256

        self.img_cols = 256

        self.channels = 1

        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        self.latent_dim = 100



        optimizer = Adam(0.0002, 0.5)



        # Build and compile the discriminator

        self.discriminator = self.build_discriminator()

        self.discriminator.compile(loss='binary_crossentropy',

            optimizer=optimizer,

            metrics=['accuracy'])



        # Build the generator

        self.generator = self.build_generator()



        # The generator takes noise as input and generates imgs

        z = Input(shape=(self.latent_dim,))

        img = self.generator(z)



        # For the combined model we will only train the generator

        self.discriminator.trainable = False



        # The discriminator takes generated images as input and determines validity

        validity = self.discriminator(img)



        # The combined model  (stacked generator and discriminator)

        # Trains the generator to fool the discriminator

        self.combined = Model(z, validity)

        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)





    def build_generator(self):



        model = Sequential()



        model.add(Dense(256, input_dim=self.latent_dim))

        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(512))

        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(1024))

        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(np.prod(self.img_shape), activation='tanh'))

        model.add(Reshape(self.img_shape))



        model.summary()



        noise = Input(shape=(self.latent_dim,))

        img = model(noise)



        return Model(noise, img)



    def build_discriminator(self):



        model = Sequential()



        model.add(Flatten(input_shape=self.img_shape))

        model.add(Dense(512))

        model.add(LeakyReLU(alpha=0.2))

        model.add(Dense(256))

        model.add(LeakyReLU(alpha=0.2))

        model.add(Dense(1, activation='sigmoid'))

        model.summary()



        img = Input(shape=self.img_shape)

        validity = model(img)



        return Model(img, validity)



    def train(self, epochs, batch_size=128, sample_interval=50):



        # Load the dataset

        #(X_train, _), (_, _) = mnist.load_data()

        directory = r"C:\\users\\sarva\\Projects\\GAN_face\\person\\"

        training_data = np.empty([983,256,256])

        i = 0

        for file in os.listdir(directory):
        
            training_data[i] = np.array(ndimage.imread(directory+file, flatten=False))
            
        
            i = i + 1

        # Rescale -1 to 1

        X_train = training_data / 127.5 - 1.

        X_train = np.expand_dims(X_train, axis=3)



        # Adversarial ground truths

        valid = np.ones((batch_size, 1))

        fake = np.zeros((batch_size, 1))



        for epoch in range(epochs):



            # ---------------------

            #  Train Discriminator

            # ---------------------



            # Select a random batch of images

            idx = np.random.randint(0, X_train.shape[0], batch_size)

            imgs = X_train[idx]



            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))



            # Generate a batch of new images

            gen_imgs = self.generator.predict(noise)



            # Train the discriminator

            d_loss_real = self.discriminator.train_on_batch(imgs, valid)

            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)

            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)



            # ---------------------

            #  Train Generator

            # ---------------------



            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))



            # Train the generator (to have the discriminator label samples as valid)

            g_loss = self.combined.train_on_batch(noise, valid)



            # Plot the progress

            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))



            # If at save interval => save generated image samples

            if epoch % sample_interval == 0:

                self.sample_images(epoch)



    def sample_images(self, epoch):

        r, c = 5, 5

        noise = np.random.normal(0, 1, (r * c, self.latent_dim))

        gen_imgs = self.generator.predict(noise)



        # Rescale images 0 - 1

        gen_imgs = 0.5 * gen_imgs + 0.5



        fig, axs = plt.subplots(r, c)

        cnt = 0

        for i in range(r):

            for j in range(c):

                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')

                axs[i,j].axis('off')

                cnt += 1

        fig.savefig("GAN_images/%d.png" % epoch)

        plt.close()





if __name__ == '__main__':

    gan = GAN()

    gan.train(epochs=300, batch_size=10, sample_interval=1)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               33554944  
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 33,686,529
Trainable params: 33,686,529
Non-trainable pa

C:\Users\sarva\Anaconda\lib\site-packages\ipykernel_launcher.py:214: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


Instructions for updating:
Use tf.cast instead.


C:\Users\sarva\Anaconda\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.743513, acc.: 20.00%] [G loss: 0.702563]
1 [D loss: 1.298521, acc.: 80.00%] [G loss: 0.592189]
2 [D loss: 2.382109, acc.: 55.00%] [G loss: 0.992614]
3 [D loss: 0.413194, acc.: 80.00%] [G loss: 2.432961]
4 [D loss: 0.952221, acc.: 80.00%] [G loss: 2.844276]
5 [D loss: 0.852360, acc.: 80.00%] [G loss: 1.644858]
6 [D loss: 2.694592, acc.: 60.00%] [G loss: 2.581427]
7 [D loss: 2.069552, acc.: 70.00%] [G loss: 2.472780]
8 [D loss: 1.809710, acc.: 70.00%] [G loss: 3.651322]
9 [D loss: 0.311621, acc.: 80.00%] [G loss: 5.227997]
10 [D loss: 0.722138, acc.: 80.00%] [G loss: 3.212879]
11 [D loss: 0.543819, acc.: 85.00%] [G loss: 5.044106]
12 [D loss: 0.364367, acc.: 85.00%] [G loss: 5.759973]
13 [D loss: 0.555628, acc.: 80.00%] [G loss: 6.754730]
14 [D loss: 0.137963, acc.: 90.00%] [G loss: 4.826013]
15 [D loss: 0.564941, acc.: 85.00%] [G loss: 4.579136]
16 [D loss: 0.868876, acc.: 75.00%] [G loss: 5.488503]
17 [D loss: 1.104679, acc.: 80.00%] [G loss: 6.355289]
18 [D loss: 0.352595

148 [D loss: 0.670746, acc.: 85.00%] [G loss: 12.877521]
149 [D loss: 0.966031, acc.: 80.00%] [G loss: 10.003552]
150 [D loss: 0.128389, acc.: 90.00%] [G loss: 10.742258]
151 [D loss: 0.260562, acc.: 90.00%] [G loss: 9.963778]
152 [D loss: 1.928677, acc.: 70.00%] [G loss: 8.244992]
153 [D loss: 1.253744, acc.: 80.00%] [G loss: 8.741196]
154 [D loss: 1.728269, acc.: 80.00%] [G loss: 8.939075]
155 [D loss: 2.239958, acc.: 80.00%] [G loss: 7.777328]
156 [D loss: 0.709557, acc.: 80.00%] [G loss: 8.496914]
157 [D loss: 0.003705, acc.: 100.00%] [G loss: 10.636057]
158 [D loss: 1.956645, acc.: 75.00%] [G loss: 11.583559]
159 [D loss: 0.841290, acc.: 85.00%] [G loss: 10.039459]
160 [D loss: 1.109788, acc.: 90.00%] [G loss: 8.426184]
161 [D loss: 2.587376, acc.: 60.00%] [G loss: 14.507462]
162 [D loss: 0.890556, acc.: 80.00%] [G loss: 14.266687]
163 [D loss: 1.428982, acc.: 70.00%] [G loss: 14.506699]
164 [D loss: 1.578865, acc.: 75.00%] [G loss: 13.030795]
165 [D loss: 0.651041, acc.: 95.00%] 

292 [D loss: 0.021783, acc.: 100.00%] [G loss: 15.041229]
293 [D loss: 0.349950, acc.: 95.00%] [G loss: 16.118095]
294 [D loss: 0.574266, acc.: 95.00%] [G loss: 14.513570]
295 [D loss: 0.558528, acc.: 90.00%] [G loss: 15.592539]
296 [D loss: 0.000314, acc.: 100.00%] [G loss: 15.091593]
297 [D loss: 0.245210, acc.: 90.00%] [G loss: 14.851176]
298 [D loss: 0.522332, acc.: 85.00%] [G loss: 16.118095]
299 [D loss: 0.450131, acc.: 85.00%] [G loss: 15.068715]


In [12]:
gan.train(epochs=1, batch_size=10, sample_interval=1)

C:\Users\sarva\Anaconda\lib\site-packages\ipykernel_launcher.py:214: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
C:\Users\sarva\Anaconda\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.270708, acc.: 95.00%] [G loss: 14.506289]


In [11]:
 def sample_images(self, epoch):

        noise = np.random.normal(0, 1, (1, self.latent_dim))

        gen_imgs = self.generator.predict(noise)

        fig.savefig("GAN_images/%d.png" % epoch)

        plt.close()


In [14]:

import random

from scipy import ndarray

import skimage as sk

from skimage import transform

from skimage import util



def random_rotation(image_array: ndarray):

    # pick a random degree of rotation between 25% on the left and 25% on the right

    random_degree = random.uniform(-25, 25)

    return sk.transform.rotate(image_array, random_degree)



def random_noise(image_array: ndarray):

    # add random noise to the image

    return sk.util.random_noise(image_array)



def horizontal_flip(image_array: ndarray):

    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !

    return image_array[:, ::-1]



ImportError: cannot import name '_validate_lengths'

In [ ]:
person_pixels = []

for Pic in os.listdir('person'):
    fname = 'person/' + Pic
    image = np.array(ndimage.imread(fname, flatten=False))
    my_image = scipy.misc.imresize(image, size=(square_size, square_size)).reshape(square_size*square_size,1)
    person_pixels.append(my_image)